# import libraries

In [0]:
import pickle
import os 
import gzip

import tensorflow as tf
import numpy as np
import pandas as pd

import sklearn
from sklearn import metrics 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sn

import matplotlib.pyplot as plt

import librosa

from datetime import datetime 

from scipy import signal
from scipy.signal import spectrogram

#! pip install lspopt
#from lspopt.lsp import spectrogram_lspopt

from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, concatenate
from keras.layers import Convolution2D, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Permute, Reshape
from keras.layers import Convolution1D, Conv1D, MaxPooling1D, GlobalAveragePooling1D, Input
from keras.optimizers import Adam
from keras.utils import np_utils, plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Define model

In [0]:
 def model():
    # Input A = Raw, Input B = Spectogram
    inputA = Input(shape=(3000,1))
    inputB = Input(shape=(2, 129, 13))

    # the first branch operates on the first input Raw
    raws = BatchNormalization()(inputA)
    raws = Conv1D(filters=60, kernel_size=20, activation='relu')(raws)
    raws = MaxPooling1D(pool_size=6)(raws)
    raws = Conv1D(filters=80, kernel_size= 15, activation='relu')(raws)
    raws = MaxPooling1D(pool_size=6)(raws)
    raws = Dropout(0.2)(raws)
    raws = Conv1D(filters=80, kernel_size= 10, activation='relu')(raws)
    raws = MaxPooling1D(pool_size=6)(raws)
    raws = Conv1D(filters=80, kernel_size= 10, activation='relu')(raws)
    raws = MaxPooling1D(pool_size=2)(raws)
    raws = Dense(120, activation='relu')(raws)
    raws = GlobalAveragePooling1D()(raws)

    raws = Model(inputs = inputA, outputs = raws)

    # the second branch opreates on the second input
    spect = BatchNormalization()(inputB)
    spect = Conv2D(filters=64, kernel_size = (1,6), activation='relu')(spect)
    spect = MaxPooling2D(pool_size=(1,2))(spect)
    spect = Dropout(0.2)(spect)

    spect = BatchNormalization()(spect)
    spect = Conv2D(filters=128, kernel_size=(1,4), activation='relu')(spect)
    spect = MaxPooling2D(pool_size=(1,2))(spect)
    spect = Dropout(0.2)(spect)

    spect = BatchNormalization()(spect)
    spect = Conv2D(filters=256, kernel_size=(1,2), activation='relu')(spect)
    spect = MaxPooling2D(pool_size=(1,2))(spect)
    spect = Dropout(0.2)(spect)

    spect = BatchNormalization()(spect)
    spect = Conv2D(filters=64, kernel_size=(1,2), activation='relu')(spect)
    spect = MaxPooling2D(pool_size=(1,2))(spect)
    spect = Dropout(0.2)(spect)

    spect = GlobalAveragePooling2D()(spect)

    spect = Model(inputs = inputB, outputs = spect)

    # combine the output of the two branches
    combined = concatenate([raws.output, spect.output])

    # apply a FC layer and then a regression prediction on the combined outputs
    z = Dense(6, activation='softmax')(combined)

    # our model will accept the inputs of the two branches and outputs a vector
    model = Model(inputs=[raws.input, spect.input], outputs=z)

    return model

# Compile model and fit

In [0]:
# Reading Data
with open('/src/Data_Raw_signals.pkl', 'rb') as raw:
  train_data_raw, class_raw = pickle.load(raw)

def multi_model(batch, window_size, order, patience, num_epochs, c,
                train_data_raw = train_data_raw, model = model()):

  #Apply Savgol filter to raw signals
  train_data_raw_sav = savgol_filter(train_data_raw[:,0,:], window_size, order).reshape(-1,3000,1)

  # Getting Spectograms, and demonstrating the results
  # Change c_perameter to change resolution. lower = more resolute
  f, t, Sxx = spectrogram(train_data_raw, fs=100)
  Sxx1 = librosa.power_to_db(Sxx, ref = np.mean)

  #Split the data
  raw_train, raw_val, y_train, y_val = train_test_split(train_data_raw_sav, class_raw, test_size=0.2, random_state = 42)
  spect_train, spect_val, y_train, y_val = train_test_split(Sxx1, class_raw, test_size=0.2, random_state = 42)

  # Compile the model
  #optim = optimizers.SGD(momentum=0.9, nesterov=True)
  model.compile(loss='sparse_categorical_crossentropy', 
                metrics=['sparse_categorical_accuracy'], 
                optimizer='adam')

  #Fitting model
  num_epochs = num_epochs
  num_batch_size = batch

  checkpointer = [ModelCheckpoint(filepath= "/out/mixed_mod_simplesect_1draw_otherchannel.hdf5", 
                                verbose=1, save_best_only=True),
                  EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience = patience)]

  start = datetime.now()

  history = model.fit([raw_train, spect_train], y_train, 
            batch_size=num_batch_size, 
            epochs=num_epochs, 
            validation_data=([raw_val, spect_val], y_val), 
            callbacks=checkpointer, 
            verbose=1)

  duration = datetime.now() - start
  print("Training completed in time: ", duration)

  # Obtaining accuracy  
  model.load_weights("/out/mixed_mod_simplesect_1draw_otherchannel.hdf5")
  _, val_acc = model.evaluate([raw_val, spect_val], y_val)

  return val_acc, raw_val, spect_val, y_val, model

val_acc, raw_val, spect_val, y_val, model = multi_model(12, 13, 4, c = 2.5, 
                                                        num_epochs = 200, 
                                                        patience = 10)

# Perform prediction and obtain metrics

In [0]:
# Start Here
# Getting metrics
model.load_weights("/out/mixed_mod_simplesect_1draw_otherchannel.hdf5")

y_pred = model.predict([raw_val, spect_val], batch_size= 1, verbose=1)
y_pred = np.argmax(y_pred, axis=1)
y_true = y_val

classes_labels = ['class R', 'class 1', 'class 2', 'class 3', 'class 4', 'class W']

np.set_printoptions(precision=2, suppress='true')
cm_norm = confusion_matrix(y_true, y_pred, normalize='true')

# Plot of normalized confusion matrix 
df_cm_norm = pd.DataFrame(cm_norm, classes_labels, classes_labels)
sn.set(font_scale=1.4) # for label size
sn.heatmap(df_cm_norm, annot=True, annot_kws={"size": 12}) # font size
plt.savefig('/out/Confusion_matrix.png')
plt.show()

print(classification_report(y_true, y_pred, target_names=classes_labels))

# Tο ΤΧΤ
# Reading test data
with open('/src/Test_Raw_signals_no_labels.pkl', 'rb') as test:
  test_data_raw = pickle.load(test)

test_data_raw = test_data_raw[0]

# Filtering test data
test_data_raw_sav = savgol_filter(test_data_raw[:,0,:], 13, 4).reshape(-1, 3000, 1)

#Exporting Spectograms from raw data
f, t, Spect_test = spectrogram(test_data_raw, fs=100)
spect_test = librosa.power_to_db(Spect_test, ref = np.mean)

model.load_weights("/out/mixed_mod_simplesect_1draw_otherchannel.hdf5")

prediction_result = model.predict([test_data_raw_sav, spect_test], batch_size= 12, verbose=1)
prediction_result_arg = np.argmax(prediction_result, axis = 1)
prediction_label = pd.DataFrame(prediction_result_arg, columns=["Stage"])

test = pd.DataFrame(test_data_raw_sav[:, 0, 0])
final = pd.concat([test, prediction_label], axis = 1) #1
final.to_csv('/out/answer.csv', index=False)